In [41]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.datasets import mnist


## Básico

In [ ]:
x = np.arange(0, 10)

In [ ]:
# cria Dataset object à partir de um np array
dx = tf.data.Dataset.from_tensor_slices(x)

In [ ]:
iterator = iter(dx)

In [ ]:
for i in range(10):
    print(next(iterator).numpy())

In [ ]:
# garante que sempre irá pegar um valor do dataset, utilizando o repeat()
dx = tf.data.Dataset.from_tensor_slices(x).repeat()
iterator = iter(dx)

In [ ]:
for i in range(15):
    print(next(iterator).numpy())

In [ ]:
# pega batches do dataset
dx = tf.data.Dataset.from_tensor_slices(x).repeat().batch(3)
iterator = iter(dx)

In [ ]:
for i in range(15):
    print(next(iterator).numpy())

In [ ]:
def simple_zip_example():
    x = np.arange(0, 10)
    y = np.arange(1, 11)
    
    # cria objetos dataset à partir dos arrays
    dx = tf.data.Dataset.from_tensor_slices(x)
    dy = tf.data.Dataset.from_tensor_slices(y)
    
    # faz uma tupla de valores e labels
    dcomb = tf.data.Dataset.zip((dx, dy)).repeat().batch(3)
    iterator = iter(dcomb)
    
    for i in range(15):
        data = next(iterator)
        print(f"x:{data[0].numpy()}, y:{data[1].numpy()}")

simple_zip_example()

## MNIST DATASET

In [34]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [42]:
# converte valores para float e depois aplica normalização
dx_train = tf.data.Dataset.from_tensor_slices(x_train).map(lambda x: tf.cast(x, tf.float32) / 255.0)

#transforma em um array 3D altura, largura e os canais de cor(1 porque são imagens em escala de cinza)
dx_train = dx_train.map(lambda x: tf.reshape(x, (28, 28, 1)))

# transforma labels em int
dy_train = tf.data.Dataset.from_tensor_slices(y_train).map(lambda x: tf.cast(x, tf.int32))

In [43]:
# junta a imagem com um label, faz um shuffle nos dados e pega uma amostra de 32 imagens
train_dataset = tf.data.Dataset.zip((dx_train, dy_train)).repeat().shuffle(500).batch(32)

In [44]:
# converte valores para float e depois aplica normalização
dx_test = tf.data.Dataset.from_tensor_slices(x_test).map(lambda x: tf.cast(x, tf.float32) / 255.0)

#transforma em um array 3D altura, largura e os canais de cor(1 porque são imagens em escala de cinza)
dx_test = dx_test.map(lambda x: tf.reshape(x, (28, 28, 1)))

# transforma labels em int
dy_test = tf.data.Dataset.from_tensor_slices(y_test).map(lambda x: tf.cast(x, tf.int32))

In [45]:
# junta a imagem com um label, faz um shuffle nos dados e pega uma amostra de 32 imagens
test_dataset = tf.data.Dataset.zip((dx_test, dy_test)).repeat().shuffle(1000).batch(500)

In [46]:
# cria iterators

train_iter = iter(train_dataset)
test_iter = iter(test_dataset)

In [48]:
class ConvLayer(tf.keras.layers.Layer):
    def __init__(self, activation, input_channels, output_channels, window_size, pool_size, filt_stride, pool_stride,
        initializer=tf.keras.initializers.he_normal()):
        super(ConvLayer, self).__init__()
        self.initializer = initializer
        self.activation = activation
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.window_size = window_size
        self.pool_size = pool_size
        self.filt_stride = filt_stride
        self.pool_stride = pool_stride
        self.w = self.add_weight(shape=(window_size[0], window_size[1], input_channels, output_channels),
                                 initializer=self.initializer,trainable=True)
        
        self.b = self.add_weight(shape=(output_channels,), initializer=tf.zeros_initializer, trainable=True)
    
    def call(self, inputs):
        filt_stride = [1, self.filt_stride[0], self.filt_stride[1], 1]
        out_layer = tf.nn.conv2d(inputs, self.w, filt_stride, padding='SAME')
        # add the bias
        out_layer += self.b
        out_layer = self.activation(out_layer)
        pool_shape = [1, self.pool_size[0], self.pool_size[1], 1]
        pool_strides = [1, self.pool_stride[0], self.pool_stride[1], 1]
        out_layer = tf.nn.max_pool(out_layer, ksize=pool_shape, strides=pool_strides, padding='SAME')
        return out_layer

In [51]:
def loss_fn(logits, labels):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = labels, 
                                                                           logits= logits))
    return cross_entropy

In [49]:
model = tf.keras.Sequential([
    ConvLayer(tf.nn.relu, 1, 32, [5, 5], [2, 2], [1, 1], [2, 2]),
    ConvLayer(tf.nn.relu, 32, 64, [5, 5], [2, 2], [1, 1], [2, 2]),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(300, activation=tf.nn.relu, kernel_initializer=tf.keras.initializers.he_normal()),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation=None)
])

In [53]:

iterations = 1000
optimizer = tf.keras.optimizers.Adam()
for i in range(iterations):
    batch_x, batch_y = next(train_iter)
    batch_y_one_hot = tf.one_hot(batch_y, 10)
    
    with tf.GradientTape() as tape:
        logits = model(batch_x)
        loss = loss_fn(logits, batch_y_one_hot)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    if i % 50 == 0:
        max_idxs = tf.argmax(logits, axis=1)
        train_acc = np.sum(max_idxs.numpy() == batch_y.numpy()) / len(batch_y)
        # get the test data
        x_test, y_test = next(test_iter)
        
        test_logits = model(x_test, training=False)
        max_idxs = tf.argmax(test_logits, axis=1)
        test_acc = np.sum(max_idxs.numpy() == y_test.numpy()) / len(y_test)
        
        print(f"Iter: {i}, loss={loss:.3f}, "
              f"train accuracy={train_acc * 100:.3f}%, test accuracy={test_acc * 100:.3f}%")

Iter: 0, loss=2.741, train accuracy=3.125%, test accuracy=24.200%
Iter: 50, loss=0.019, train accuracy=100.000%, test accuracy=88.200%
Iter: 100, loss=0.141, train accuracy=96.875%, test accuracy=93.600%
Iter: 150, loss=0.067, train accuracy=100.000%, test accuracy=94.200%
Iter: 200, loss=0.505, train accuracy=81.250%, test accuracy=93.200%
Iter: 250, loss=0.287, train accuracy=90.625%, test accuracy=94.800%
Iter: 300, loss=0.017, train accuracy=100.000%, test accuracy=92.800%
Iter: 350, loss=0.168, train accuracy=96.875%, test accuracy=96.200%
Iter: 400, loss=0.036, train accuracy=100.000%, test accuracy=97.800%
Iter: 450, loss=0.168, train accuracy=96.875%, test accuracy=98.800%
Iter: 500, loss=0.019, train accuracy=100.000%, test accuracy=98.200%
Iter: 550, loss=0.093, train accuracy=96.875%, test accuracy=98.600%
Iter: 600, loss=0.175, train accuracy=96.875%, test accuracy=98.800%
Iter: 650, loss=0.069, train accuracy=96.875%, test accuracy=98.200%
Iter: 700, loss=0.133, train accu